In [1]:
using Flux
using Flux: Data.DataLoader
using Flux: onehotbatch,onecold,crossentropy
using Flux: @epochs
using Statistics
using MLDatasets

In [23]:
x_train_temp,y_train_temp = MNIST.traindata();
x_test_temp,y_test_temp = MNIST.testdata();

In [24]:
println("The size of the training images are $(size(x_train_temp)) and the size of training labels is $(size(y_train_temp))")
print("The size of the testing images are $(size(x_test_temp)) and the size of testing labels is $(size(y_test_temp))")

The size of the training images are (28, 28, 60000) and the size of training labels is (60000,)
The size of the testing images are (28, 28, 10000) and the size of testing labels is (10000,)

In [30]:
x_train = Flux.unsqueeze(x_train_temp,3)
x_test = Flux.unsqueeze(x_test_temp,3)

y_train = onehotbatch(y_train_temp,0:9)
y_test = onehotbatch(y_test_temp,0:9)

#Displaying the updated shape of the elements

println("The size of the training images are $(size(x_train)) and the size of training labels is $(size(y_train))")
print("The size of the testing images are $(size(x_test)) and the size of testing labels is $(size(y_test))")


The size of the training images are (28, 28, 1, 60000) and the size of training labels is (10, 60000)
The size of the testing images are (28, 28, 1, 10000) and the size of testing labels is (10, 10000)

In [74]:
layer1 = Conv((3,3),1=>8,relu,stride=2)
layer2 = Conv((3,3),8=>16,relu)
layer3 = Conv((3,3),16=>32,relu)
layer4 = GlobalMaxPool()
layer5 = flatten
layer6 = Dense(32,10)
layer7 = softmax;

In [127]:
layers = [layer1, layer2, layer3, layer4, layer5, layer6, layer7]
x = x_train[:,:,:,1:2]

for l in layers
    println("The layer is $l")
    println("The size of the input layer is $(size(x))")
    x = l(x)
    println("The size of the output layer is $(size(x))")
    println("-"^30)
end

The layer is Conv((3, 3), 1=>8, relu)
The size of the input layer is (28, 28, 1, 2)
The size of the output layer is (13, 13, 8, 2)
------------------------------
The layer is Conv((3, 3), 8=>16, relu)
The size of the input layer is (13, 13, 8, 2)
The size of the output layer is (11, 11, 16, 2)
------------------------------
The layer is Conv((3, 3), 16=>32, relu)
The size of the input layer is (11, 11, 16, 2)
The size of the output layer is (9, 9, 32, 2)
------------------------------
The layer is GlobalMaxPool()
The size of the input layer is (9, 9, 32, 2)
The size of the output layer is (1, 1, 32, 2)
------------------------------
The layer is flatten
The size of the input layer is (1, 1, 32, 2)
The size of the output layer is (32, 2)
------------------------------
The layer is Dense(32, 10)
The size of the input layer is (32, 2)
The size of the output layer is (10, 2)
------------------------------
The layer is softmax
The size of the input layer is (10, 2)
The size of the output la

In [140]:
model = Chain(layer1, layer2, layer3, layer4, layer5,layer6, layer7)
# model = Chain(Dense(784,32),Dense(32,10),softmax)
L(x,y) = Flux.crossentropy(model(x),y)
opt = Flux.Optimise.ADAM()
ps = Flux.params(model)

# evalcb() = @show(L(x_train, y_train))
# throttled_cb = Flux.throttle(evalcb, 5)

# callback() = push!(loss_vector, L(x_train, y_train))
# loss_vector = Vector{Float32}()
print("Training started")
Flux.train!(L,ps,zip(x_train[:,:,:,1:2],y_train[:,1:2]),opt, cb = Flux.throttle(() -> print("training"),1))

Training started

LoadError: MethodError: no method matching (::Conv{2,4,typeof(relu),Array{Float32,4},Array{Float32,1}})(::FixedPointNumbers.Normed{UInt8,8})
Closest candidates are:
  Any(!Matched::AbstractArray) at C:\Users\Saad.LAKES\.julia\packages\Flux\q3zeA\src\layers\conv.jl:142

In [130]:
size(y_train[:,1:2])

(10, 2)

In [139]:
model=nothing

In [114]:
foo=reshape(x_train_temp,(784,60000));

In [117]:
size(x_train_temp)

(28, 28, 60000)

In [120]:
preprocess(p) = vec(Float64.(p))
foo = preprocess.(x_train_temp[1:100])

LoadError: MethodError: no method matching vec(::Float64)
Closest candidates are:
  vec(!Matched::LinearAlgebra.Transpose{T,var"#s828"} where var"#s828"<:(AbstractArray{T,1} where T) where T) at C:\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.5\LinearAlgebra\src\adjtrans.jl:226
  vec(!Matched::LinearAlgebra.Adjoint{var"#s828",var"#s8281"} where var"#s8281"<:(AbstractArray{T,1} where T) where var"#s828"<:Real) at C:\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.5\LinearAlgebra\src\adjtrans.jl:227
  vec(!Matched::SparseArrays.AbstractSparseArray{Tv,Ti,1} where Ti where Tv) at C:\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.5\SparseArrays\src\sparsevector.jl:911
  ...

In [126]:
length.(x_train_temp)

28×28×60000 Array{Int64,3}:
[:, :, 1] =
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1 